In [ ]:
// purpose of this notebook:
// to find coins that have the highest likelihood of going up with the least amount of new capital inflows

// data we need:
// 1. total coin market cap ovet time
// 2. a list of coins
// 3. the market cap of those coins over

// what we do with that data:
// * convert the coin market caps of each coin into a dominance %
// * calculate the time since the last the coin last hit peak dominance
// * multiply the dominance % by the time since the last peak
// * sort the coins in descending order of the product of the dominance % and the time since the last peak

#r "nuget: FSharp.Data"
#r "nuget: Microsoft.Data.Analysis"

open FSharp.Data

type BinanceExchangeInfo = JsonProvider<"https://api.binance.com/api/v3/exchangeInfo">
type PriceStream = JsonProvider<"https://api.binance.com/api/v3/klines?symbol=1INCHDOWNUSDT&interval=1d">

let show a =
    printf "%A\n" a

let exchInfo = BinanceExchangeInfo.Load("https://api.binance.com/api/v3/exchangeInfo")

let assets = 
    exchInfo.Symbols 
    |> Array.filter (fun x-> x.QuoteAsset.IndexOf("USDT") <> -1 )
    |> Array.filter (fun x-> x.BaseAsset.IndexOf("UP") = -1 && x.BaseAsset.IndexOf("DOWN") = -1) 
    |> Array.map (fun x -> x.Symbol) 
    |> Array.sort

type Candle = 
    { Time : uint64
    ; Open : decimal
    ; High:  decimal
    ; Low : decimal
    ; Close: decimal }
    
let makeCandle (assetRow:(decimal array)) =
    { Time = (uint64)assetRow.[0]
    ; Open = assetRow.[1]
    ; High = assetRow.[2]
    ; Low = assetRow.[3]
    ; Close = assetRow.[4] }

let assetHistory symbol (limit:uint64) =
    PriceStream.Load("https://api.binance.com/api/v3/klines?symbol=" + symbol + "&interval=1d&limit=" + limit.ToString())
    |> Array.map makeCandle

let ethHistory = 
    assetHistory "ETHUSDT" 1000UL

let ethPricedAssetHistory limit symbol = 
    let asset = assetHistory symbol limit
    let eth = 
        ethHistory
        |> Array.skip (ethHistory.Length - asset.Length)

    asset
    |> Array.zip eth
    |> Array.map (fun (ethRow, assetRow) -> 
        { Time = assetRow.Time
        ; Open = assetRow.Open / ethRow.Open
        ; High = assetRow.High / ethRow.High
        ; Low = assetRow.Low / ethRow.Low
        ; Close = assetRow.Close / ethRow.Close })

let ath assetHistory = 
    assetHistory 
    |> Array.maxBy (fun x -> x.High)

let timeSinceLastAth now assetHistory =
    let lastAth = (ath assetHistory).Time
    now - lastAth

let daysSinceLastAth now assetHistory =
    (timeSinceLastAth now assetHistory) / (24UL * 60UL * 60UL) / 1000UL

"BTCUSDT" |> (ethPricedAssetHistory 1000UL) |> (daysSinceLastAth ethHistory.[ethHistory.Length - 1].Time)

type assetAthDays = 
    { Asset: string
    ; DaysSinceLastAth: uint64 }

let allDaysSinceLastAth limit assetList = 
    assetList
    |> Array.Parallel.map (fun asset -> 
        show asset
        { Asset = (string(asset).Substring(0, string(asset).IndexOf("USDT")))
        // ; PriceNow = ethPricedAssetHistory 
        ; DaysSinceLastAth = asset 
            |> ethPricedAssetHistory limit 
            |> daysSinceLastAth ethHistory.[ethHistory.Length - 1].Time})

assets  
|> Array.take assets.Length
|> allDaysSinceLastAth 1000UL
|> Array.sortBy (fun x -> x.DaysSinceLastAth)
//|> Array.skip 20



Installed Packages FSharp.Data, 4.2.3 Microsoft.Data.Analysis, 0.18.0

"TRXUSDT"
"ATOMUSDT"
"1INCHUSDT"
"C98USDT"
"DAIUSDT"
"EGLDUSDT"
"AUDIOUSDT"
"HARDUSDT"
"AAVEUSDT"
"KEEPUSDT"
"PHAUSDT"
"NMRUSDT"
"ELFUSDT"
"KEYUSDT"
"TUSDUSDT"
"WAXPUSDT"
"NPXSUSDT"
"STRAXUSDT"
"WAVESUSDT"
"YFIIUSDT"
"DASHUSDT"
"FIOUSDT"
"FIROUSDT"
"HBARUSDT"
"STXUSDT"
"YFIUSDT"
"RSRUSDT"
"BELUSDT"
"ROSEUSDT"
"MBLUSDT"
"CAKEUSDT"
"PNTUSDT"
"MATICUSDT"
"BEARUSDT"
"ACMUSDT"
"AUDUSDT"
"BKRWUSDT"
"CELOUSDT"
"DATAUSDT"
"ENJUSDT"
"FISUSDT"
"HCUSDT"
"KLAYUSDT"
"EOSBEARUSDT"
"KMDUSDT"
"WINGUSDT"
"KSMUSDT"
"WNXMUSDT"
"FLMUSDT"
"ADAUSDT"
"OCEANUSDT"
"MBOXUSDT"
"NULSUSDT"
"POLSUSDT"
"TVKUSDT"
"SUNUSDT"
"RUNEUSDT"
"DCRUSDT""WTCUSDT"
"RVNUSDT"
"BLZUSDT"

"FORTHUSDT"
"POLYUSDT"
"AUTOUSDT"
"CELRUSDT"
"HIVEUSDT"
"SUSDUSDT"
"HOTUSDT"
"MCOUSDT"
"DENTUSDT"
"CHRUSDT"
"CKBUSDT"
"BNBBULLUSDT"
"OGNUSDT"
"BNTUSDT"
"AKROUSDT"
"SXPUSDT"
"AVAXUSDT"
"TWTUSDT"
"EOSUSDT"
"PSGUSDT"
"YGGUSDT"
"ALICEUSDT"
"UNFIUSDT"
"SCUSDT"
"BADGERUSDT"
"ZENUSDT"
"DFUSDT"
"MDXUSDT"
"ZRXUSDT"
"ALPHAUSDT"
"BALUSDT"
"BTCSTUSDT"
"COCOSUS

index,Asset,DaysSinceLastAth
0,AXS,0
1,DYDX,3
2,FIDA,3
3,DF,8
4,SYS,8
5,LUNA,8
6,YGG,9
7,ILV,10
8,MLN,11
9,ELF,16
